# Model
Create neural network using Keras & TensorFlow

## Import Packages

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
def smooth_L1_loss(y_true, y_pred):
    THRESHOLD = K.variable(1.0)
    mae = K.abs(y_true - y_pred)
    flag = K.greater(mae, THRESHOLD)
    loss = K.mean(K.switch(flag, (mae - 0.5), K.pow(mae, 2)), axis=-1)
    return loss

In [3]:
X_train = pd.read_csv('processed data/predictors_train.csv', index_col = 0)
y_train = pd.read_csv('processed data/target_train.csv', names = ['price'])

X_test = pd.read_csv('processed data/predictors_test.csv', index_col = 0)
y_test = pd.read_csv('processed data/target_test.csv', names = ['price'])

In [4]:
#convert testing/training sets to numpy arrays
X_train = np.array(X_train.values)
y_train = np.array(y_train.values)

X_test = np.array(X_test.values)
y_test = np.array(y_test.values)

In [7]:
#reshape data for input into the LSTM model
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [6]:
sequence_length = 17  # number of days before predicting
n_features = 32  # number of features in the data set
time_step = 1  # number of samples

In [8]:
model = Sequential()
#1 layer LSTM with 500 nodes and Xavier (glorot_uniform) activation and L1 loss function
model.add(LSTM(500, input_shape=(sequence_length, time_step, n_features),
                       kernel_initializer='glorot_uniform', stateful=True))
model.add(Dense(1))
model.compile(loss='smooth_L1_loss', optimizer='adam')

#fit model to training data
model.fit(X_train, y_train)

ValueError: Input 0 is incompatible with layer lstm_1: expected ndim=3, found ndim=4

In [ ]:
prd = model.predict(X_test)